<a href="https://colab.research.google.com/github/Moeez774/Cyber-Attack-Detector/blob/master/chapters/04-chat-memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/langchain-course/blob/main/chapters/04-chat-memory.ipynb)

### LangChain Essentials

# Conversational Memory for OpenAI - LangChain

Conversational memory allows our chatbots and agents to remember previous interactions within a conversation. Without conversational memory, our chatbots would only ever be able to respond to the last message they received, essentially forgetting all previous messages with each new message.

Naturally, conversations require our chatbots to be able to respond over multiple interactions and refer to previous messages to understand the context of the conversation.

In [1]:
!pip install -qU \
  langchain-core==0.3.33 \
  langchain-openai==0.3.3 \
  langchain-community==0.3.16 \
  langsmith==0.3.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.3/333.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.3 MB/s eta 0:00:00


In [2]:
!pip install --upgrade "pydantic==2.10.6"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.4 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.33.2
    Uninstalling pydantic_core-2.33.2:
      Successfully uninstalled pydantic_core-2.33.2
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.11.7
    Uninstalling pydantic-2.11.7:
      Successfully uninstalled pydantic-2.11.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mcp 1.13.1 requires pydantic<3.0.0,>=2.11.0, but you have pydantic 2.10.6 which is incompatible.


---

> ⚠️ We will be using OpenAI for this example allowing us to run everything via API. If you would like to use Ollama instead, check out the [Ollama LangChain Course](https://github.com/aurelio-labs/langchain-course/tree/main/notebooks/ollama).

---

---

> ⚠️ If using LangSmith, add your API key below:

In [3]:
import os
from getpass import getpass

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") or \
    getpass("Enter LangSmith API Key: ")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "aurelioai-langchain-course-chat-memory-openai"

Enter LangSmith API Key: ··········


---

## LangChain's Memory Types

LangChain versions `0.0.x` consisted of various conversational memory types. Most of these are due for deprecation but still hold value in understanding the different approaches that we can take to building conversational memory.

Throughout the notebook we will be referring to these _older_ memory types and then rewriting them using the recommended `RunnableWithMessageHistory` class. We will learn about:

* `ConversationBufferMemory`: the simplest and most intuitive form of conversational memory, keeping track of a conversation without any additional bells and whistles.
* `ConversationBufferWindowMemory`: similar to `ConversationBufferMemory`, but only keeps track of the last `k` messages.
* `ConversationSummaryMemory`: rather than keeping track of the entire conversation, this memory type keeps track of a summary of the conversation.
* `ConversationSummaryBufferMemory`: merges the `ConversationSummaryMemory` and `ConversationTokenBufferMemory` types.

We'll work through each of these memory types in turn, and rewrite each one using the `RunnableWithMessageHistory` class.

## Initialize our LLM

Before jumping into our memory types, let's initialize our LLM. We will use OpenAI's `gpt-4o-mini` model, if you need an API key you can get one from [OpenAI's website](https://platform.openai.com/settings/organization/api-keys).

In [4]:
import os
from getpass import getpass
from langchain_openai import ChatOpenAI
from google.colab import userdata

OPENROUTER_API_KEY = userdata.get("OPENROUTER_API_KEY")

# For normal accurate responses
llm = ChatOpenAI(temperature=0.0, api_key=OPENROUTER_API_KEY, base_url="https://openrouter.ai/api/v1", model="x-ai/grok-4-fast:free")

## 1. `ConversationBufferMemory`

`ConversationBufferMemory` is the simplest form of conversational memory, it is _literally_ just a place that we store messages, and then use to feed messages into our LLM.

Let's start with LangChain's original `ConversationBufferMemory` object, we are setting `return_messages=True` to return the messages as a list of `ChatMessage` objects — unless using a non-chat model we would always set this to `True` as without it the messages are passed as a direct string which can lead to unexpected behavior from chat LLMs.

In [5]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

/tmp/ipython-input-1448044083.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


There are several ways that we can add messages to our memory, using the `save_context` method we can add a user query (via the `input` key) and the AI's response (via the `output` key). So, to create the following conversation:

```
User: Hi, my name is James
AI: Hey James, what's up? I'm an AI model called Zeta.
User: I'm researching the different types of conversational memory.
AI: That's interesting, what are some examples?
User: I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.
AI: That's interesting, what's the difference?
User: Buffer memory just stores the entire conversation, right?
AI: That makes sense, what about ConversationBufferWindowMemory?
User: Buffer window memory stores the last k messages, dropping the rest.
AI: Very cool!
```

We do:

In [6]:
memory.save_context(
    {"input": "Hi, my name is James"},  # user message
    {"output": "Hey James, what's up? I'm an AI model called Zeta."}  # AI response
)
memory.save_context(
    {"input": "I'm researching the different types of conversational memory."},  # user message
    {"output": "That's interesting, what are some examples?"}  # AI response
)
memory.save_context(
    {"input": "I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory."},  # user message
    {"output": "That's interesting, what's the difference?"}  # AI response
)
memory.save_context(
    {"input": "Buffer memory just stores the entire conversation, right?"},  # user message
    {"output": "That makes sense, what about ConversationBufferWindowMemory?"}  # AI response
)
memory.save_context(
    {"input": "Buffer window memory stores the last k messages, dropping the rest."},  # user message
    {"output": "Very cool!"}  # AI response
)

Before using the memory, we need to load in any variables for that memory type — in this case, there are none, so we just pass an empty dictionary:

In [7]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
  HumanMess

With that, we've created our buffer memory. Before feeding it into our LLM let's quickly view the alternative method for adding messages to our memory. With this other method, we pass individual user and AI messages via the `add_user_message` and `add_ai_message` methods. To reproduce what we did above, we do:

In [9]:
memory = ConversationBufferMemory(return_messages=True)

memory.chat_memory.add_user_message("Hi, my name is James")
memory.chat_memory.add_ai_message("Hey James, what's up? I'm an AI model called Zeta.")
memory.chat_memory.add_user_message("I'm researching the different types of conversational memory.")
memory.chat_memory.add_ai_message("That's interesting, what are some examples?")
memory.chat_memory.add_user_message("I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.")
memory.chat_memory.add_ai_message("That's interesting, what's the difference?")
memory.chat_memory.add_user_message("Buffer memory just stores the entire conversation, right?")
memory.chat_memory.add_ai_message("That makes sense, what about ConversationBufferWindowMemory?")
memory.chat_memory.add_user_message("Buffer window memory stores the last k messages, dropping the rest.")
memory.chat_memory.add_ai_message("Very cool!")

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
  HumanMess

The outcome is exactly the same in either case. To pass this onto our LLM, we need to create a `ConversationChain` object — which is already deprecated in favor of the `RunnableWithMessageHistory` class, which we will cover in a moment.

In [10]:
from langchain.chains import ConversationChain

chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

/tmp/ipython-input-839683240.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  chain = ConversationChain(


In [11]:
chain.invoke({"input": "what is my name again?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}), AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}), HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}), HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what's the difference?", additional_kw

{'input': 'what is my name again?',
 'history': [HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}

### `ConversationBufferMemory` with `RunnableWithMessageHistory`

As mentioned, the `ConversationBufferMemory` type is due for deprecation. Instead, we can use the `RunnableWithMessageHistory` class to implement the same functionality.

When implementing `RunnableWithMessageHistory` we will use **L**ang**C**hain **E**xpression **L**anguage (LCEL) and for this we need to define our prompt template and LLM components. Our `llm` has already been defined, so now we just define a `ChatPromptTemplate` object.

In [12]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    ChatPromptTemplate
)

system_prompt = "You are a helpful assistant called Zeta."

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{query}"),
])

We can link our `prompt_template` and our `llm` together to create a pipeline via LCEL.

In [17]:
pipeline = prompt_template | llm

Our `RunnableWithMessageHistory` requires our `pipeline` to be wrapped in a `RunnableWithMessageHistory` object. This object requires a few input parameters. One of those is `get_session_history`, which requires a function that returns a `ChatMessageHistory` object based on a session ID. We define this function ourselves:

In [15]:
from langchain_core.chat_history import InMemoryChatMessageHistory

chat_map = {}
def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = InMemoryChatMessageHistory()
    return chat_map[session_id]

We also need to tell our runnable which variable name to use for the chat history (ie `history`) and which to use for the user's query (ie `query`).

In [18]:
from langchain_core.runnables.history import RunnableWithMessageHistory

pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history"
)

Now we invoke our runnable:

In [19]:
pipeline_with_history.invoke({
    "query": "Hi my name is James"
}, config={ "session_id": "abc_123" })

AIMessage(content='Hi James! Nice to meet you. How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 130, 'total_tokens': 225, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 80, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 113}}, 'model_name': 'x-ai/grok-4-fast:free', 'system_fingerprint': 'fp_9362061f30', 'finish_reason': 'stop', 'logprobs': None}, id='run-e465056b-90a7-466e-a117-67242e557e3a-0', usage_metadata={'input_tokens': 130, 'output_tokens': 95, 'total_tokens': 225, 'input_token_details': {'audio': 0, 'cache_read': 113}, 'output_token_details': {'reasoning': 80}})

Our chat history will now be memorized and retrieved whenever we invoke our runnable with the same session ID.

In [20]:
pipeline_with_history.invoke(
    {"query": "What is my name again?"},
    config={"session_id": "abc_123"}
)

AIMessage(content='Your name is James! What else can I assist you with?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 159, 'total_tokens': 250, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 78, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 113}}, 'model_name': 'x-ai/grok-4-fast:free', 'system_fingerprint': 'fp_9362061f30', 'finish_reason': 'stop', 'logprobs': None}, id='run-acb33858-a59c-413b-8438-6e3cb46725a6-0', usage_metadata={'input_tokens': 159, 'output_tokens': 91, 'total_tokens': 250, 'input_token_details': {'audio': 0, 'cache_read': 113}, 'output_token_details': {'reasoning': 78}})

We have now recreated the `ConversationBufferMemory` type using the `RunnableWithMessageHistory` class. Let's continue onto other memory types and see how these can be implemented.

## 2. `ConversationBufferWindowMemory`

The `ConversationBufferWindowMemory` type is similar to `ConversationBufferMemory`, but only keeps track of the last `k` messages. There are a few reasons why we would want to keep only the last `k` messages:

* More messages mean more tokens are sent with each request, more tokens increases latency _and_ cost.

* LLMs tend to perform worse when given more tokens, making them more likely to deviate from instructions, hallucinate, or _"forget"_ information provided to them. Conciseness is key to high performing LLMs.

* If we keep _all_ messages we will eventually hit the LLM's context window limit, by adding a window size `k` we can ensure we never hit this limit.

The buffer window solves many problems that we encounter with the standard buffer memory, while still being a very simple and intuitive form of conversational memory.

In [21]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=4, return_messages=True)

/tmp/ipython-input-3216785012.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=4, return_messages=True)


We populate this memory using the same methods as before:

In [22]:
memory.chat_memory.add_user_message("Hi, my name is James")
memory.chat_memory.add_ai_message("Hey James, what's up? I'm an AI model called Zeta.")
memory.chat_memory.add_user_message("I'm researching the different types of conversational memory.")
memory.chat_memory.add_ai_message("That's interesting, what are some examples?")
memory.chat_memory.add_user_message("I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.")
memory.chat_memory.add_ai_message("That's interesting, what's the difference?")
memory.chat_memory.add_user_message("Buffer memory just stores the entire conversation, right?")
memory.chat_memory.add_ai_message("That makes sense, what about ConversationBufferWindowMemory?")
memory.chat_memory.add_user_message("Buffer window memory stores the last k messages, dropping the rest.")
memory.chat_memory.add_ai_message("Very cool!")

memory.load_memory_variables({})

{'history': [HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer window memory stores the last k messages, dropping the rest.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Very cool!', additional_kwargs={}, response_metadata={})]}

As before, we use the `ConversationChain` object (again, this is deprecated and we will rewrite it with `RunnableWithMessageHistory` in a moment).

In [23]:
chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

Now let's see if our LLM remembers our name:

In [24]:
chain.invoke({"input": "what is my name again?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}), HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}), HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}), AIMessage(content='That makes sense, what about 

{'input': 'what is my name again?',
 'history': [HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer window memory stores the last k messages, dropping the rest.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Very cool!', additional_kw

The reason our LLM can no longer remember our name is because we have set the `k` parameter to `4`, meaning that only the last messages are stored in memory, as we can see above this does not include the first message where we introduced ourselves.

Based on the agent forgetting our name, we might wonder _why_ we would ever use this memory type compared to the standard buffer memory. Well, as with most things in AI, it is always a trade-off. Here we are able to support much longer conversations, use less tokens, and improve latency — but these come at the cost of forgetting non-recent messages.

### `ConversationBufferWindowMemory` with `RunnableWithMessageHistory`

To implement this memory type using the `RunnableWithMessageHistory` class, we can use the same approach as before. We define our `prompt_template` and `llm` as before, and then wrap our pipeline in a `RunnableWithMessageHistory` object.

For the window feature, we need to define a custom version of the `InMemoryChatMessageHistory` class that removes any messages beyond the last `k` messages.

In [43]:
from pydantic import BaseModel, Field
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage

class BufferWindowMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    k: int = Field(default_factory=int)

    def __init__(self, k: int):
      super().__init__(k=k)

    def add_messages(self, messages: list[BaseMessage]):

      self.messages.extend(messages)
      self.messages = self.messages[-self.k:]

    def clear(self):
      self.messages = []

In [44]:
chat_map = {}
def get_chat_history(session_id: str, k: int = 4) -> BufferWindowMessageHistory:
  if session_id not in chat_map:
    chat_map[session_id] = BufferWindowMessageHistory(k=k)
  return chat_map[session_id]

In [45]:
from langchain_core.runnables import ConfigurableFieldSpec

pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="k",
            annotation=int,
            name="k",
            description="The number of messages to keep in the history",
            default=4,
        )
    ]
)

Now we invoke our runnable, this time passing a `k` parameter via the `config` parameter.

In [46]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is James"},
    config={"configurable": {"session_id": "id_k4", "k": 4}}
)

AIMessage(content='Hi James! Nice to meet you. How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 97, 'prompt_tokens': 131, 'total_tokens': 228, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 82, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 113}}, 'model_name': 'x-ai/grok-4-fast:free', 'system_fingerprint': 'fp_9362061f30', 'finish_reason': 'stop', 'logprobs': None}, id='run-5ae54c04-1e8d-4632-b44a-0ec058b2a22c-0', usage_metadata={'input_tokens': 131, 'output_tokens': 97, 'total_tokens': 228, 'input_token_details': {'audio': 0, 'cache_read': 113}, 'output_token_details': {'reasoning': 82}})

We can also modify the messages that are stored in memory by modifying the records inside the `chat_map` dictionary directly.

In [47]:
chat_map["id_k4"].clear()  # clear the history

# manually insert history
chat_map["id_k4"].add_user_message("Hi, my name is James")
chat_map["id_k4"].add_ai_message("I'm an AI model called Zeta.")
chat_map["id_k4"].add_user_message("I'm researching the different types of conversational memory.")
chat_map["id_k4"].add_ai_message("That's interesting, what are some examples?")
chat_map["id_k4"].add_user_message("I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.")
chat_map["id_k4"].add_ai_message("That's interesting, what's the difference?")
chat_map["id_k4"].add_user_message("Buffer memory just stores the entire conversation, right?")
chat_map["id_k4"].add_ai_message("That makes sense, what about ConversationBufferWindowMemory?")
chat_map["id_k4"].add_user_message("Buffer window memory stores the last k messages, dropping the rest.")
chat_map["id_k4"].add_ai_message("Very cool!")

chat_map["id_k4"].messages

[HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Buffer window memory stores the last k messages, dropping the rest.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Very cool!', additional_kwargs={}, response_metadata={})]

Now let's see at which `k` value our LLM remembers our name — from the above we can already see that with `k=4` our name is not mentioned, so when running with `k=4` we should expect the LLM to forget our name:

In [48]:
pipeline_with_history.invoke(
    {"query": "what is my name again?"},
    config={"configurable": {"session_id": "id_k4", "k": 4}}
)

AIMessage(content="I'm sorry, but you haven't shared your name with me in this conversation. If you'd like to tell me, I'd be happy to remember it going forward! What's your name?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 233, 'prompt_tokens': 184, 'total_tokens': 417, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 198, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 120}}, 'model_name': 'x-ai/grok-4-fast:free', 'system_fingerprint': 'fp_9362061f30', 'finish_reason': 'stop', 'logprobs': None}, id='run-5a8d50a5-8ad8-4717-a5d9-3381fbc8ed26-0', usage_metadata={'input_tokens': 184, 'output_tokens': 233, 'total_tokens': 417, 'input_token_details': {'audio': 0, 'cache_read': 120}, 'output_token_details': {'reasoning': 198}})

Now let's initialize a new session with `k=14`.

In [49]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is James"},
    config={"session_id": "id_k14", "k": 14}
)

AIMessage(content='Hi James! Nice to meet you. How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 131, 'total_tokens': 239, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 93, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 130}}, 'model_name': 'x-ai/grok-4-fast:free', 'system_fingerprint': 'fp_9362061f30', 'finish_reason': 'stop', 'logprobs': None}, id='run-2d9d276b-dd71-40a0-8be5-9c0660aa3358-0', usage_metadata={'input_tokens': 131, 'output_tokens': 108, 'total_tokens': 239, 'input_token_details': {'audio': 0, 'cache_read': 130}, 'output_token_details': {'reasoning': 93}})

We'll manually insert the remaining messages as before:

In [50]:
chat_map["id_k14"].add_user_message("I'm researching the different types of conversational memory.")
chat_map["id_k14"].add_ai_message("That's interesting, what are some examples?")
chat_map["id_k14"].add_user_message("I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.")
chat_map["id_k14"].add_ai_message("That's interesting, what's the difference?")
chat_map["id_k14"].add_user_message("Buffer memory just stores the entire conversation, right?")
chat_map["id_k14"].add_ai_message("That makes sense, what about ConversationBufferWindowMemory?")
chat_map["id_k14"].add_user_message("Buffer window memory stores the last k messages, dropping the rest.")
chat_map["id_k14"].add_ai_message("Very cool!")

chat_map["id_k14"].messages

[HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi James! Nice to meet you. How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 131, 'total_tokens': 239, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 93, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 130}}, 'model_name': 'x-ai/grok-4-fast:free', 'system_fingerprint': 'fp_9362061f30', 'finish_reason': 'stop', 'logprobs': None}, id='run-2d9d276b-dd71-40a0-8be5-9c0660aa3358-0', usage_metadata={'input_tokens': 131, 'output_tokens': 108, 'total_tokens': 239, 'input_token_details': {'audio': 0, 'cache_read': 130}, 'output_token_details': {'reasoning': 93}}),
 HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_meta

In [51]:
pipeline_with_history.invoke(
    {"query": "what is my name again?"},
    config={"session_id": "id_k14", "k": 14}
)

AIMessage(content="Your name is James! We've been chatting about conversational memory types—anything else on your mind?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 146, 'prompt_tokens': 266, 'total_tokens': 412, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 127, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 113}}, 'model_name': 'x-ai/grok-4-fast:free', 'system_fingerprint': 'fp_9362061f30', 'finish_reason': 'stop', 'logprobs': None}, id='run-7a91ff8b-5aa3-43b1-a946-c492c54d7249-0', usage_metadata={'input_tokens': 266, 'output_tokens': 146, 'total_tokens': 412, 'input_token_details': {'audio': 0, 'cache_read': 113}, 'output_token_details': {'reasoning': 127}})

In [52]:
chat_map["id_k14"].messages

[HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi James! Nice to meet you. How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 131, 'total_tokens': 239, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 93, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 130}}, 'model_name': 'x-ai/grok-4-fast:free', 'system_fingerprint': 'fp_9362061f30', 'finish_reason': 'stop', 'logprobs': None}, id='run-2d9d276b-dd71-40a0-8be5-9c0660aa3358-0', usage_metadata={'input_tokens': 131, 'output_tokens': 108, 'total_tokens': 239, 'input_token_details': {'audio': 0, 'cache_read': 130}, 'output_token_details': {'reasoning': 93}}),
 HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_meta

That's it! We've rewritten our buffer window memory using the recommended `RunnableWithMessageHistory` class.

## 3. `ConversationSummaryMemory`

Next up we have `ConversationSummaryMemory`, this memory type keeps track of a summary of the conversation rather than the entire conversation. This is useful for long conversations where we don't need to keep track of the entire conversation, but we do want to keep some thread of the full conversation.

As before, we'll start with the original memory class before reimplementing it with the `RunnableWithMessageHistory` class.

In [53]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(llm=llm)

/tmp/ipython-input-988334424.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm)


Unlike with the previous memory types, we need to provide an `llm` to initialize `ConversationSummaryMemory`. The reason for this is that we need an LLM to generate the conversation summaries.

Beyond this small tweak, using `ConversationSummaryMemory` is the same as with our previous memory types when using the deprecated `ConversationChain` object.

In [54]:
chain = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

Let's test:

In [55]:
chain.invoke({"input": "hello there my name is James"})
chain.invoke({"input": "I am researching the different types of conversational memory."})
chain.invoke({"input": "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory."})
chain.invoke({"input": "Buffer memory just stores the entire conversation"})
chain.invoke({"input": "Buffer window memory stores the last k messages, dropping the rest."})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hello there my name is James
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human introduces himself as James. The AI introduces itself as Grok, built by xAI and inspired by the Hitchhiker's Guide to the Galaxy and JARVIS from Iron Man, and asks what's on his mind.
Human: I am researching the different types of conversational memory.
AI:

> Finished chain.


> Ent

{'input': 'Buffer window memory stores the last k messages, dropping the rest.',
 'history': "New summary:\nThe human introduces himself as James. The AI introduces itself as Grok, built by xAI and inspired by the Hitchhiker's Guide to the Galaxy and JARVIS from Iron Man, and asks what's on his mind. James says he's researching different types of conversational memory. Grok provides a detailed overview of the main types—short-term (working) memory via context windows, long-term memory using external stores like vector embeddings, implicit (parametric) memory from training data, and hybrid/multi-modal memory integrating sensory inputs—covering how they work, strengths, limitations, and examples like ChatGPT, RAG systems, and MemGPT. Grok summarizes the field's evolution and challenges (e.g., privacy, accuracy) and asks what sparked James's interest, such as building a chatbot or academic work. James mentions he's been looking at ConversationBufferMemory and ConversationBufferWindowMemor

We can see how the conversation summary varies with each new message. Let's see if the LLM is able to recall our name:

In [56]:
chain.invoke({"input": "What is my name again?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human introduces himself as James. The AI introduces itself as Grok, built by xAI and inspired by the Hitchhiker's Guide to the Galaxy and JARVIS from Iron Man, and asks what's on his mind. James says he's researching different types of conversational memory. Grok provides a detailed overview of the main types—short-term (working) memory via context windows, long-term memory using external stores like vector embeddings, implicit (parametric) memory from training data, and hybrid/multi-modal memory integrating sensory inputs—covering how they work, strengths, limitations, and examples like ChatGPT, RAG systems, and MemGPT. Grok summarizes the field's evolution and c

{'input': 'What is my name again?',
 'history': "The human introduces himself as James. The AI introduces itself as Grok, built by xAI and inspired by the Hitchhiker's Guide to the Galaxy and JARVIS from Iron Man, and asks what's on his mind. James says he's researching different types of conversational memory. Grok provides a detailed overview of the main types—short-term (working) memory via context windows, long-term memory using external stores like vector embeddings, implicit (parametric) memory from training data, and hybrid/multi-modal memory integrating sensory inputs—covering how they work, strengths, limitations, and examples like ChatGPT, RAG systems, and MemGPT. Grok summarizes the field's evolution and challenges (e.g., privacy, accuracy) and asks what sparked James's interest, such as building a chatbot or academic work. James mentions he's been looking at ConversationBufferMemory and ConversationBufferWindowMemory. Grok explains these as foundational LangChain short-term

As this information was stored in the summary the LLM successfully recalled our name. This may not always be the case, by summarizing the conversation we inevitably compress the full amount of information and so we may lose key details occasionally. Nonetheless, this is a great memory type for long conversations while retaining some key information.

### `ConversationSummaryMemory` with `RunnableWithMessageHistory`

Let's implement this memory type using the `RunnableWithMessageHistory` class. As with the window buffer memory, we need to define a custom implementation of the `InMemoryChatMessageHistory` class. We'll call this one `ConversationSummaryMessageHistory`.

In [58]:
from langchain_core.messages import SystemMessage


class ConversationSummaryMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    llm: ChatOpenAI = Field(default_factory=ChatOpenAI)

    def __init__(self, llm: ChatOpenAI):
        super().__init__(llm=llm)

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add messages to the history, removing any messages beyond
        the last `k` messages.
        """
        self.messages.extend(messages)
        # construct the summary chat messages
        summary_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(
                "Given the existing conversation summary and the new messages, "
                "generate a new summary of the conversation. Ensuring to maintain "
                "as much relevant information as possible."
            ),
            HumanMessagePromptTemplate.from_template(
                "Existing conversation summary:\n{existing_summary}\n\n"
                "New messages:\n{messages}"
            )
        ])
        # format the messages and invoke the LLM
        new_summary = self.llm.invoke(
            summary_prompt.format_messages(
                existing_summary=self.messages.content,
                messages=[x.content for x in messages]
            )
        )
        # replace the existing history with a single system summary message
        self.messages = [SystemMessage(content=new_summary.content)]

    def clear(self) -> None:
        """Clear the history."""
        self.messages = []

In [59]:
chat_map = {}
def get_chat_history(session_id: str, llm: ChatOpenAI) -> ConversationSummaryMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = ConversationSummaryMessageHistory(llm=llm)
    # return the chat history
    return chat_map[session_id]

In [60]:
pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="llm",
            annotation=ChatOpenAI,
            name="LLM",
            description="The LLM to use for the conversation summary",
            default=llm,
        )
    ]
)

Now we invoke our runnable, this time passing a `llm` parameter via the `config` parameter.

In [61]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is James"},
    config={"session_id": "id_123", "llm": llm}
)

AIMessage(content="Hi James! Nice to meet you. I'm Zeta, your helpful assistant. What can I do for you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 98, 'prompt_tokens': 131, 'total_tokens': 229, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 75, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 130}}, 'model_name': 'x-ai/grok-4-fast:free', 'system_fingerprint': 'fp_9362061f30', 'finish_reason': 'stop', 'logprobs': None}, id='run-d8956b72-df0b-4608-b96f-7764cc38c18a-0', usage_metadata={'input_tokens': 131, 'output_tokens': 98, 'total_tokens': 229, 'input_token_details': {'audio': 0, 'cache_read': 130}, 'output_token_details': {'reasoning': 75}})

Let's see what summary was generated:

In [62]:
chat_map["id_123"].messages

[HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hi James! Nice to meet you. I'm Zeta, your helpful assistant. What can I do for you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 98, 'prompt_tokens': 131, 'total_tokens': 229, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 75, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 130}}, 'model_name': 'x-ai/grok-4-fast:free', 'system_fingerprint': 'fp_9362061f30', 'finish_reason': 'stop', 'logprobs': None}, id='run-d8956b72-df0b-4608-b96f-7764cc38c18a-0', usage_metadata={'input_tokens': 131, 'output_tokens': 98, 'total_tokens': 229, 'input_token_details': {'audio': 0, 'cache_read': 130}, 'output_token_details': {'reasoning': 75}})]

Let's continue the conversation and see if the summary is updated:

In [ ]:
pipeline_with_history.invoke(
    {"query": "I'm researching the different types of conversational memory."},
    config={"session_id": "id_123", "llm": llm}
)

chat_map["id_123"].messages

[HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi James! Nice to meet you. How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 98, 'prompt_tokens': 131, 'total_tokens': 229, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 83, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 112}}, 'model_name': 'x-ai/grok-4-fast:free', 'system_fingerprint': 'fp_9362061f30', 'finish_reason': 'stop', 'logprobs': None}, id='run-7a5ffaf0-771f-4a62-9549-cc9682d94f81-0', usage_metadata={'input_tokens': 131, 'output_tokens': 98, 'total_tokens': 229, 'input_token_details': {'audio': 0, 'cache_read': 112}, 'output_token_details': {'reasoning': 83}}),
 HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metada

So far so good! Let's continue with a few more messages before returning to the name question.

In [ ]:
for msg in [
    "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory.",
    "Buffer memory just stores the entire conversation",
    "Buffer window memory stores the last k messages, dropping the rest."
]:
    pipeline_with_history.invoke(
        {"query": msg},
        config={"session_id": "id_123", "llm": llm}
    )

Let's see the latest summary:

In [ ]:
chat_map["id_123"].messages

[HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi James! Nice to meet you. How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 98, 'prompt_tokens': 131, 'total_tokens': 229, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 83, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 112}}, 'model_name': 'x-ai/grok-4-fast:free', 'system_fingerprint': 'fp_9362061f30', 'finish_reason': 'stop', 'logprobs': None}, id='run-7a5ffaf0-771f-4a62-9549-cc9682d94f81-0', usage_metadata={'input_tokens': 131, 'output_tokens': 98, 'total_tokens': 229, 'input_token_details': {'audio': 0, 'cache_read': 112}, 'output_token_details': {'reasoning': 83}}),
 HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metada

The information about our name has been maintained, so let's see if this is enough for our LLM to correctly recall our name.

In [ ]:
pipeline_with_history.invoke(
    {"query": "What is my name again?"},
    config={"session_id": "id_123", "llm": llm}
)

AIMessage(content="James! You introduced yourself at the start of our chat. See how conversational memory keeps things connected? If there's more you'd like to dive into about memory types or implementations, I'm all ears.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 219, 'prompt_tokens': 2943, 'total_tokens': 3162, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 181, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 2784}}, 'model_name': 'x-ai/grok-4-fast:free', 'system_fingerprint': 'fp_9362061f30', 'finish_reason': 'stop', 'logprobs': None}, id='run-778aa5cc-ba40-45c7-8487-3ce04683b454-0', usage_metadata={'input_tokens': 2943, 'output_tokens': 219, 'total_tokens': 3162, 'input_token_details': {'audio': 0, 'cache_read': 2784}, 'output_token_details': {'reasoning': 181}})

Perfect! We've successfully implemented the `ConversationSummaryMemory` type using the `RunnableWithMessageHistory` class.

## 4. `ConversationSummaryBufferMemory`

Our final memory type acts as a combination of `ConversationSummaryMemory` and `ConversationBufferMemory`. It keeps the buffer for the conversation up until the previous `n` tokens, anything beyond that limit is summarized then dropped from the buffer. Producing something like:


```
# ~~ a summary of previous interactions
The user named James introduced himself and the AI responded, introducing itself as an AI model called Zeta.
James then said he was researching the different types of conversational memory and Zeta asked for some
examples.
# ~~ the most recent messages
Human: I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory.
AI: That's interesting, what's the difference?
Human: Buffer memory just stores the entire conversation
AI: That makes sense, what about ConversationBufferWindowMemory?
Human: Buffer window memory stores the last k messages, dropping the rest.
AI: Very cool!
```


In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=300,
    return_messages=True
)

<ipython-input-43-ed1cccbb366f>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


As before, we set up the deprecated memory type using the `ConversationChain` object.

In [ ]:
chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

First invoke with a single message:

In [ ]:
chain.invoke({"input": "Hi, my name is James"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: Hi, my name is James
AI:

> Finished chain.


{'input': 'Hi, my name is James',
 'history': [HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hello, James! It's great to meet you! How's your day going so far?", additional_kwargs={}, response_metadata={})],
 'response': "Hello, James! It's great to meet you! How's your day going so far?"}

Looks good so far, let's continue with a few more messages:

In [ ]:
for msg in [
    "I'm researching the different types of conversational memory.",
    "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory.",
    "Buffer memory just stores the entire conversation",
    "Buffer window memory stores the last k messages, dropping the rest."
]:
    chain.invoke({"input": msg})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello, James! It's great to meet you! How's your day going so far?", additional_kwargs={}, response_metadata={})]
Human: I'm researching the different types of conversational memory.
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[SystemMessage(content='The human, nam

We can see with each new message the initial `SystemMessage` is updated with a new summary of the conversation. This initial `SystemMessage` is then followed by the most recent `AIMessage` and `HumanMessage` objects.

### `ConversationSummaryBufferMemory` with `RunnableWithMessageHistory`

As with the previous memory types, we will implement this memory type again using the `RunnableWithMessageHistory` class. In our implementation we will modify the buffer window to be based on the number of messages rather than number of tokens. This tweak will make our implementation more closely aligned with original buffer window.

We will implement all of this via a new `ConversationSummaryBufferMessageHistory` class.

In [63]:
class ConversationSummaryBufferMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    llm: ChatOpenAI = Field(default_factory=ChatOpenAI)
    k: int = Field(default_factory=int)

    def __init__(self, llm: ChatOpenAI, k: int):
        super().__init__(llm=llm, k=k)

    def add_messages(self, messages: list[BaseMessage]) -> None:

      existing_summary: SystemMessage | None = None
      old_messages: list[BaseMessage] = []

      if len(self.messages) > 0 and isinstance(self.messages[0], SystemMessage):
        existing_summary = self.messages.pop(0)

      self.messages.extend(messages)

      if len(self.messages) > self.k:

        old_messages = self.messages[:-self.k]
        self.messages = self.messages[-self.k:]

      if old_messages is None:
        return

      summary_prompt = ChatPromptTemplate.from_messages([
                SystemMessagePromptTemplate.from_template(
                    "Given the existing conversation summary and the new messages, "
                    "generate a new summary of the conversation. Ensuring to maintain "
                    "as much relevant information as possible."
                ),
                HumanMessagePromptTemplate.from_template(
                    "Existing conversation summary:\n{existing_summary}\n\n"
                    "New messages:\n{old_messages}"
                )
            ])

      new_summary = self.llm.invoke(
          summary_prompt.format_messages(
              existing_summary=existing_summary,
              old_messages=[x.content for x in old_messages]
          )
      )

      self.messages = [SystemMessage(content=new_summary.content)] + self.messages

    def clear(self) -> None:
        """Clear the history."""
        self.messages = []

Redefine the `get_chat_history` function to use our new `ConversationSummaryBufferMessageHistory` class.

In [64]:
chat_map = {}
def get_chat_history(session_id: str, llm: ChatOpenAI, k: int) -> ConversationSummaryBufferMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = ConversationSummaryBufferMessageHistory(llm=llm, k=k)
    # return the chat history
    return chat_map[session_id]

Setup our pipeline with new configurable fields.

In [65]:
pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="llm",
            annotation=ChatOpenAI,
            name="LLM",
            description="The LLM to use for the conversation summary",
            default=llm,
        ),
        ConfigurableFieldSpec(
            id="k",
            annotation=int,
            name="k",
            description="The number of messages to keep in the history",
            default=4,
        )
    ]
)

Finally, we invoke our runnable:

In [66]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is James"},
    config={"session_id": "id_123", "llm": llm, "k": 4}
)
chat_map["id_123"].messages

[SystemMessage(content='None', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi James! Nice to meet you. How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 97, 'prompt_tokens': 131, 'total_tokens': 228, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 82, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 113}}, 'model_name': 'x-ai/grok-4-fast:free', 'system_fingerprint': 'fp_9362061f30', 'finish_reason': 'stop', 'logprobs': None}, id='run-62243ee0-01a7-4f73-a061-7cfcef9c8d17-0', usage_metadata={'input_tokens': 131, 'output_tokens': 97, 'total_tokens': 228, 'input_token_details': {'audio': 0, 'cache_read': 113}, 'output_token_details': {'reasoning': 82}})]

In [68]:
for i, msg in enumerate([
    "I'm researching the different types of conversational memory.",
    "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory.",
    "Buffer memory just stores the entire conversation",
    "Buffer window memory stores the last k messages, dropping the rest."
]):
    print(f"---\nMessage {i+1}\n---\n")
    pipeline_with_history.invoke(
        {"query": msg},
        config={"session_id": "id_123", "llm": llm, "k": 4}
    )

---
Message 1
---

---
Message 2
---

---
Message 3
---

---
Message 4
---



There we go, we've successfully implemented the `ConversationSummaryBufferMemory` type using `RunnableWithMessageHistory`!